In [62]:
import re
import pandas as pd
import string
from langdetect import detect, DetectorFactory
from tqdm.notebook import tqdm

In [63]:
# Load dataset and inspect

df = pd.read_csv('../data/kyrgyz_coal_dataset_cleaned.csv')

print("Dataset shape:", df.shape)
print("Columns:", df.columns.tolist())

df.head()

Dataset shape: (2924, 19)
Columns: ['post_url', 'post_text', 'post_label', 'number_of_likes', 'number_of_shares', 'number_of_comments', 'post_date', 'comment_text', 'comment_date', 'comment_reactions_count', 'comment_mode_types', 'post_target_region', 'post_language', 'comment_language', 'coal_topic/event_tag', 'season', 'comment_sentiment_label', 'sentiment_confidence', 'news_page_name']


,post_url,post_text,post_label,number_of_likes,number_of_shares,number_of_comments,post_date,comment_text,comment_date,comment_reactions_count,comment_mode_types,post_target_region,post_language,comment_language,coal_topic/event_tag,season,comment_sentiment_label,sentiment_confidence,news_page_name
0,https://www.facebook.com/AzattykPlus/posts/252...,"Электр жетишсиз, көмүр кымбат - өкмөт келсин",negative,0,0,0,2008-09-04,NaN,NaN,NaN,NaN,countrywide,Kyrgyz,NaN,high price,fall,NaN,NaN,Azattyk
1,https://www.facebook.com/AzattykPlus/posts/283...,ТҮШТҮКТӨ КӨМҮРДҮН ТОННАСЫ 7 МИҢ СОМГО ЧЫКТЫ,negative,0,0,0,2008-09-14,NaN,NaN,NaN,NaN,Osh,Kyrgyz,NaN,high price,fall,NaN,NaN,Azattyk
2,https://www.facebook.com/AzattykPlus/posts/214...,Кызыл-Кыя шаарындагы цемент заводунун жанында ...,negative,0,1,0,2011-10-11,NaN,NaN,NaN,NaN,Batken,Kyrgyz,NaN,illegal coal mining,fall,NaN,NaN,Azattyk
3,https://www.facebook.com/AliToktakunov/posts/1...,Кызыл-Кыя шаарындагы цемент заводунун жанында ...,negative,1,0,1,2011-10-11,"Бийликтегилер карабаса ушундай болотта, бардык...",2011-10-11,0.0,text,Batken,Kyrgyz,Kyrgyz,illegal coal mining,fall,negative,high,Azattyk
4,https://www.facebook.com/groups/araba.kg/perma...,Кыргызстан.Соңку бир апта ичинде Кыргызстанда ...,negative,0,0,0,2011-08-23,NaN,NaN,NaN,NaN,countrywide,Kyrgyz,NaN,high price,summer,NaN,NaN,araba.kg-avtobazar


## Data preparation

### 1. Cleaning Post and Comment Language

In [64]:
# Ensure consistent and reproducible language detection
DetectorFactory.seed = 0

# Define detection function
def detect_lang(text):
    try:
        return detect(str(text))
    except:
        return "unknown"

# Apply language detection to both post and comment texts
tqdm.pandas()

# Detect language in post_text
df['post_lang_detect'] = df['post_text'].progress_apply(detect_lang)

# Detect language in comment_text
df['comment_lang_detect'] = df['comment_text'].progress_apply(detect_lang)

# View results
print("Post text language distribution:")
print(df['post_lang_detect'].value_counts())

print("\nComment text language distribution:")
print(df['comment_lang_detect'].value_counts())

# Optionally show samples misclassified as Russian
print("\nExample of comments detected as Russian:")
display(df[df['comment_lang_detect'] == 'ru'][['comment_text']].sample(5, random_state=1))

print("\nExample of comments detected as Russian:")
display(df[df['post_lang_detect'] == 'ru'][['post_text']].sample(5, random_state=1))

print("\nExample of comments detected as English:")
display(df[df['comment_lang_detect'] == 'en'][['comment_text']].sample(5, random_state=1))

print("\nExample of comments detected as Unknown:")
display(df[df['comment_lang_detect'] == 'unknown'][['comment_text']].sample(5, random_state=1))

  0%|          | 0/2924 [00:00<?, ?it/s]

  0%|          | 0/2924 [00:00<?, ?it/s]

Post text language distribution:
post_lang_detect
ru    2896
mk      17
bg      11
Name: count, dtype: int64

Comment text language distribution:
comment_lang_detect
ru         2094
tl          599
mk           78
bg           66
en           39
unknown      25
uk           21
tr            2
Name: count, dtype: int64

Example of comments detected as Russian:


,comment_text
2428,"Ит, авалай(.уро).берет кербен журо берет"
2467,"карапайым, эл бири бири менен кырылышбаса болду"
2516,Кайран Кыргыз калкым..чан жуткан газ жуткан ко...
924,Эч кандай крышасы жок эле иштетип жатышат жуму...
2716,Шайлоодон кийин коробуз ценаны.шайлоо рекламасы



Example of comments detected as Russian:


,post_text
2273,Көмүр! чыккан Нарында отун тартыш Кара-Кече ке...
1411,"Быйыл, өлкөдө көмүр тартыштыгы орун албайбы? Б..."
1469,Ош! облусунда 30 жерге арзандатылган көмүр сат...
204,Жумгалдын Арал аймагындагы Коко-Мерен дарыясын...
2660,ЖУМГАЛДАГЫ. МИҢ-КУШ АЙЫЛЫНЫН ТУРГУНДАРЫ АЙМАКТ...



Example of comments detected as English:


,comment_text
458,thumb down
2393,"thumb, up"
507,yuqoridegilaga insof bersin ollohim ilohim🤲🤲
1642,thumb up
2202,thumb! up



Example of comments detected as Unknown:


,comment_text
1614,:(
1606,"👍,"
1758,:(
306,:(
2289,"👍,"


In [65]:
# Filter for manually labeled Kyrgyz content
df_ky = df[
    (df['comment_language'] == 'Kyrgyz') |
    (df['post_language'] == 'Kyrgyz')
].copy()

# Kyrgyz posts
posts_df = df_ky[df_ky['post_language'] == 'Kyrgyz'][['post_text', 'post_label']].dropna()
posts_df = posts_df.rename(columns={'post_text': 'text', 'post_label': 'label'})
posts_df['source'] = 'post'

# Kyrgyz comments
comments_df = df_ky[df_ky['comment_language'] == 'Kyrgyz'][['comment_text', 'comment_sentiment_label']].dropna()
comments_df = comments_df.rename(columns={'comment_text': 'text', 'comment_sentiment_label': 'label'})
comments_df['source'] = 'comment'

# Show shapes to verify
print("Posts shape:", posts_df.shape)
print("Comments shape:", comments_df.shape)

# Only combine if there's data
combined_df = pd.concat([posts_df, comments_df], ignore_index=True)

print("Shape after Kyrgyz-only filtering:", combined_df.shape)
combined_df.sample(5) if not combined_df.empty else print("Combined DataFrame is empty.")

Posts shape: (2819, 3)
Comments shape: (2162, 3)
Shape after Kyrgyz-only filtering: (4981, 3)


,text,label,source
4245,Алла. раазы болсун ушундай балдар коп болсун а...,positive,comment
2649,Кыргызстанда эн кымбат кемур Баткен облусунда...,negative,post
2405,"Ат-Башыда, көмүр көздөн учту Ушул тапта Ат-Баш...",negative,post
262,Кара-Кече көмүр кенинин коопсуздук кызматынын ...,negative,post
4178,Сыр. эмес го мен деле айылда жашайм андай кара...,negative,comment


### 2. Removing Emojis, Punctuation and Stop Words

In [68]:
import re

# Step 1: Kyrgyz stopword list (100 words)
kyrgyz_stopwords = [
    "мен", "сен", "ал", "биз", "силер", "алар", "өзү", "менин", "сенин", "анын",
    "биздин", "силердин", "алардын", "ким", "эмне", "качан", "кандай", "канча", "кайда", "эмнеге",
    "бул", "ушул", "ошол", "андан", "анда", "ушундай", "ошондой", "ошондо", "ушондо", "ушунчалык",
    "да", "де", "же", "жана", "дагы", "эле", "эми", "анан", "бир", "эки",
    "үч", "төрт", "беш", "он", "жүз", "мың", "жыл", "күн", "ай", "саат", "өз", "тиги", 
    "менен", "үчүн", "менде", "сенде", "анда", "андагы", "менде", "сиз", "сиздер", "сиздин",
    "болуп", "болот", "болсо", "болгону", "болбогон", "экен", "экенин", "экенсиң", "экенбиз", "экенсиңер",
    "экенсиз", "жок", "бар", "ар", "бирок", "анткени", "ошондуктан", "тарабынан", "тараптан",
    "турган", "жаткан", "жөнүндө", "жөнүнөн", "бер", "алды", "алдына", "аркасынан", "ийин", "кийин", "айрым",
    "кийинки", "башка", "бири", "эч", "эч ким", "жокко", "жокту", "катар", "кылган", "чейин", "өкүлдөрү", 
    "бүгүнкү", "карата", "атат", "кайсы", "деп", "сизге", "дейт", "а", "учурда", "эх", "айтып", "м"
]

# Step 2: Define cleaning function
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation/emojis
    text = re.sub(r'http\S+|www.\S+', '', text)  # remove URLs
    text = re.sub(r'\s+', ' ', text).strip()  # normalize whitespace

    # Remove stopwords
    words = text.split()
    words = [word for word in words if word not in kyrgyz_stopwords]
    return ' '.join(words)

# Step 3: Apply cleaning
combined_df['text_clean'] = combined_df['text'].apply(clean_text)

# Step 4: Remove very short texts (less than 5 words)
combined_df['word_count'] = combined_df['text_clean'].apply(lambda x: len(x.split()))
filtered_df = combined_df[combined_df['word_count'] >= 5].copy()

# Drop helper column
filtered_df.drop(columns=['word_count'], inplace=True)

# Final shape and preview
print("Final dataset shape after cleaning:", filtered_df.shape)
filtered_df.sample(15)

Final dataset shape after cleaning: (4484, 4)


,text,label,source,text_clean
2632,Бишкек: чектөө менен сатылган арзан көмүр Суу...,negative,post,бишкек чектөө сатылган арзан көмүр суук түшкөн...
51,Жогорку Кеңеш Жогорку соттун милдетин аткарып ...,negative,post,жогорку кеңеш жогорку соттун милдетин аткарып ...
2270,Бишкекке! арзан көмүр жеткирилүүдө “Кара-Кечед...,positive,post,бишкекке арзан көмүр жеткирилүүдө каракечеден ...
200,Жумгал районунда аз камсыз уй-булелерге жалпы ...,positive,post,жумгал районунда аз камсыз уйбулелерге жалпы 1...
888,Кара-Кечеден көмүр тарткан жеке ишкер айдоочул...,negative,post,каракечеден көмүр тарткан жеке ишкер айдоочула...
2213,"Көмүр, ташыган айдоочулар иш таштады Бишкек жы...",positive,post,көмүр ташыган айдоочулар иш таштады бишкек жыл...
2610,Нарындын. Ак-Талаа районунда көмүргө байланышт...,negative,post,нарындын акталаа районунда көмүргө байланыштуу...
3222,Жегичтер 2дуйно жакшылык корбой калгыла укум т...,negative,comment,жегичтер 2дуйно жакшылык корбой калгыла укум т...
3370,Жакшы иш кылып атыпсынар бирок комурдун баасы ...,positive,comment,жакшы иш кылып атыпсынар комурдун баасы канчадан
1376,Алай районунун Согонду айылынын тургундары Та...,negative,post,алай районунун согонду айылынын тургундары тай...


### 3. Tokenization and Light Stemming

In [83]:
# Step: Tokenization (whitespace-based)
filtered_df['tokens'] = filtered_df['text_clean'].apply(lambda x: x.split())

# Optional: Light suffix stripping for common Kyrgyz endings (pseudo-stemming)
def pseudo_stem(tokens):
    suffixes = [
    "лар", "лер", "лор", "лөр",
    "тар", "тер", "тор", "төр",
    "дар", "дер", "дор", "дөр",  # plural
    "дын", "дин", "дун", "дүн", 
    "тан", "тен", "дан", "ден",
    "нан", "нен", "тон", "дон", # ablative case
    "нын", "нин", "нун", "нүн",  # possessive
    "га", "ге", "ка", "ке", "го", "на", "гө",     # dative case
    "да", "де", "та", "те", "до", "то", "дө",    # locative case
    "ды", "ди", "ту", "тү", "дү",    # accusative
    "ны", "ни", "ну", "нү", "ду",     # object
    "луу", "лүү", "лу", "лү",    # derivation (e.g., жумуш+туу)
    "сыз", "сүз", "суз", "сү",   # negative (e.g., пайдасыз)
    "чыл", "чил", "кыч", "гүч",  # agentive, group
    "чык", "чек", "чүк", "күчүк",# diminutives/abstract
    "мак", "мек", "пыз",  "чы", "бей", "ган",   # verbal noun/infinity
    "дык", "дик", "тук", "түк",  # nominalization
    "чылык", "чилик", "дагы",           # nominalizer
    "лөрдү", "дары", "ып", "оп", "пей"
               ]

    stemmed = []
    for word in tokens:
        for suf in suffixes:
            if word.endswith(suf) and len(word) > len(suf) + 2:
                word = word[: -len(suf)]
                break
        stemmed.append(word)
    return stemmed

# Apply optional stemmer
filtered_df['tokens_stemmed'] = filtered_df['tokens'].apply(pseudo_stem)

# Preview
filtered_df[['text_clean', 'tokens', 'tokens_stemmed']].sample(5)

,text_clean,tokens,tokens_stemmed
3552,болбойт бекер чыккан комурдун неси кымбат болсун,"[болбойт, бекер, чыккан, комурдун, неси, кымба...","[болбойт, бекер, чыккан, комур, неси, кымбат, ..."
412,алай районунун согонду айылынын тургундары тай...,"[алай, районунун, согонду, айылынын, тургундар...","[алай, району, согон, айылы, тургун, тайгакташ..."
3052,ушу устудо тойбос тордун айынан елбиз азап чекти,"[ушу, устудо, тойбос, тордун, айынан, елбиз, а...","[ушу, усту, тойбос, тор, айы, елбиз, азап, чекти]"
660,кара алтындын өмүрдү уурдаган түйшүгү ташкөмүр...,"[кара, алтындын, өмүрдү, уурдаган, түйшүгү, та...","[кара, алтын, өмүр, уурда, түйшүгү, ташкөмүр, ..."
2706,жогорку кеңеш жогорку соттун милдетин аткарып ...,"[жогорку, кеңеш, жогорку, соттун, милдетин, ат...","[жогорку, кеңеш, жогорку, соттун, милдетин, ат..."


### 4. Label Encoding

In [85]:
# Get unique labels
print("Unique sentiment labels:", filtered_df['label'].unique())

# Remove rows with unknown label
filtered_df = filtered_df[filtered_df['label'] != 'unknown'].copy()

# Map labels to integers
label_mapping = {
    'negative': 0,
    'neutral': 1,
    'positive': 2
}

# Apply mapping
filtered_df['label_encoded'] = filtered_df['label'].map(label_mapping)

# Verify
filtered_df[['label', 'label_encoded']].sample(15)

Unique sentiment labels: ['negative' 'unknown' 'positive' 'neutral']


,label,label_encoded
1359,negative,0
2419,positive,2
3325,negative,0
3984,negative,0
3594,negative,0
1334,negative,0
3681,negative,0
4639,negative,0
1125,negative,0
4519,negative,0


In [86]:
#filtered_df.to_csv("../data/filtered_df_stemmed.csv", index=False)

### 5. Handling Class Imbalance

In [59]:
filtered_df['label_encoded'].value_counts(normalize=True)

label_encoded
0    0.592968
2    0.294118
1    0.112914
Name: proportion, dtype: float64

In [60]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(class_weight='balanced')

In [61]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

y = filtered_df['label_encoded']

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y),
    y=y
)

print(dict(enumerate(class_weights)))

{0: 0.5621436716077537, 1: 2.9520958083832336, 2: 1.1333333333333333}
